In [ ]:
import logging
import os

from instagrapi import Client
from instagrapi.exceptions import LoginRequired
from typing import List, Dict, Literal

logger = logging.getLogger()

In [ ]:
ACCOUNT_USERNAME = os.getenv("ACCOUNT_USERNAME")
ACCOUNT_PASSWORD = os.getenv("ACCOUNT_PASSWORD")

In [ ]:
# cl = Client()
# cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD)
# cl.dump_settings("session.json")

In [ ]:
def login_user(
    USERNAME: str = ACCOUNT_USERNAME,
    PASSWORD: str = ACCOUNT_PASSWORD,
    delay_range: List[int] = [1, 3],
):
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """

    cl = Client()
    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info("Session is invalid, need to login via username and password")

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)
            login_via_session = True
            cl.delay_range = delay_range
            return cl
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info("Attempting to login via username and password. username: %s" % USERNAME)
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

In [ ]:
cl = login_user()

In [ ]:
def get_media_info(media):
    return {
        "date": media.taken_at or "",
        "location": media.location or "",
        "caption": media.caption_text or "",
        "comment_count": media.comment_count or 0,
        "like_count": media.like_count or 0,
        "play_count": media.play_count or 0,
    }

def get_all_media_info(medias):
    return [get_media_info(media) for media in medias]

In [ ]:
""" 
refer to this api link: https://subzeroid.github.io/instagrapi/usage-guide/hashtag.html
"""
def get_media_by_hashtags(
    cl,
    hashtags: List[str],
    classification: Literal["recent", "top"] = "top",
    amount: int = 100,
):
    media_dict = dict()
    for hashtag in hashtags:
        # get the top hashtags
        if classification == "top":
            media = cl.hashtag_medias_top(hashtag, amount)
        else:
            media = cl.hashtag_medias_recent(hashtag, amount)
        # get the relevant info from media
        all_hashtag_info = get_all_media_info(media)
        media_dict[hashtag] = all_hashtag_info
    return media_dict

In [ ]:
media_dict = get_media_by_hashtags(
    cl,
    ["tokyo", "paris"],
    amount=1,
)

In [ ]:
media_dict